In [54]:
import os
from datetime import datetime
import pandas as pd
from IPython.display import display
import pdfplumber
import re

In [55]:
pdf_path = r"./relat_utilizacao_20250806_203010.pdf"

assert os.path.exists(pdf_path), f"Arquivo não encontrado: {os.path.abspath(pdf_path)}"
print("OK:", os.path.abspath(pdf_path))

OK: c:\Users\leopa\Documents\Projetos\2E\Clínica Santa Saúde\clinicasantasaude\relat_utilizacao_20250806_203010.pdf


In [56]:
import re
import pdfplumber
import pandas as pd
from IPython.display import display

def normalize_line(s: str) -> str:
    if not s:
        return ""
    return re.sub(r"\s+", " ", s).strip()

RX_SEP1 = re.compile(r'N.? ?Guia ?Operad\.:\s*(\d+)\s+Benefici[áa]rio:\s*([0-9]+)\s*-\s*(.+)$', re.I)
RX_SEP2 = re.compile(r'Senha:\s*([0-9]+).*?Data\s+Solicit\.:\s*(\d{2}/\d{2}/\d{4})', re.I)

# Reconhece início de linha de tabela
RX_ROW_START = re.compile(r'^\s*\d+\s+\d{2}\s+\d+\s*-\s*', re.I)
# Reconhece final de linha completa (tem valor monetário no fim)
RX_MONEY_END = re.compile(r'R\$\s*[\d\.,]+\s*$', re.I)

def parse_row_buffer(buf: str):
    """
    Parser que trata casos onde "R$" se separa e vira apenas "$" com números espalhados
    """
    s = normalize_line(buf)

    # 1) Extrai seq, tabela, procedimento_codigo do início
    m_inicio = re.match(r'^\s*(\d+)\s+(\d{2})\s+(\d{8})', s)
    if not m_inicio:
        return None
    
    seq = int(m_inicio.group(1))
    tabela = m_inicio.group(2)
    proc_cod = m_inicio.group(3)
    
    # 2) Tenta extrair valor normal primeiro (R$ junto)
    m_valor = re.search(r'R\$\s*([\d\.,]+)', s)
    if m_valor:
        valor = float(m_valor.group(1).replace(".", "").replace(",", "."))
        return {
            "seq": seq,
            "tabela": tabela,
            "procedimento_codigo": proc_cod,
            "valor_exec": valor,
        }
    
    # 3) Se não achou "R$", procura por "$" isolado e reconstrói o valor
    m_cifrao = re.search(r'\$', s)
    if m_cifrao:
        print(f"  [DEBUG] Cifrão isolado encontrado em: {s}")
        # Pega tudo após o "$"
        parte_pos_cifrao = s[m_cifrao.end():]
        print(f"  [DEBUG] Parte após $: '{parte_pos_cifrao}'")
        
        # Estratégia: procurar padrão números + vírgula + números
        # Ou se não tiver vírgula, juntar os números sequenciais
        
        # Primeiro tenta encontrar vírgula e pegar números ao redor
        if ',' in parte_pos_cifrao:
            # Tem vírgula - pega números antes e depois da primeira vírgula
            pos_virgula = parte_pos_cifrao.find(',')
            antes_virgula = parte_pos_cifrao[:pos_virgula]
            depois_virgula = parte_pos_cifrao[pos_virgula+1:]
            
            nums_antes = re.findall(r'(\d)', antes_virgula)
            nums_depois = re.findall(r'(\d)', depois_virgula)
            
            print(f"  [DEBUG] Números antes da vírgula: {nums_antes}")
            print(f"  [DEBUG] Números depois da vírgula: {nums_depois}")
            
            if nums_antes and nums_depois:
                # Junta números antes e depois da vírgula
                parte_inteira = ''.join(nums_antes)
                parte_decimal = ''.join(nums_depois[:2])  # máximo 2 casas decimais
                valor_str = f"{parte_inteira},{parte_decimal}"
                print(f"  [DEBUG] Valor reconstruído: {valor_str}")
                valor = float(valor_str.replace(",", "."))
            else:
                print(f"  [DEBUG] Não conseguiu reconstruir valor com vírgula")
                return None
        else:
            # Sem vírgula - pega todos os números e assume últimos 2 são centavos
            nums = re.findall(r'(\d)', parte_pos_cifrao)
            print(f"  [DEBUG] Números encontrados (sem vírgula): {nums}")
            
            if len(nums) >= 2:
                # Assume que os últimos 2 dígitos são centavos
                parte_inteira = ''.join(nums[:-2]) if len(nums) > 2 else '0'
                parte_decimal = ''.join(nums[-2:])
                valor_str = f"{parte_inteira},{parte_decimal}"
                print(f"  [DEBUG] Valor reconstruído: {valor_str}")
                valor = float(valor_str.replace(",", "."))
            elif len(nums) == 1:
                # Apenas um número, assume centavos
                valor = float("0.0" + nums[0])
            else:
                print(f"  [DEBUG] Não conseguiu reconstruir valor")
                return None
        
        return {
            "seq": seq,
            "tabela": tabela,
            "procedimento_codigo": proc_cod,
            "valor_exec": valor,
        }
    
    # Se não encontrou nem "R$" nem "$", falha
    return None

def parse_pdf_text_as_table(pdf_path: str):
    registros = []
    falhas = []

    contexto = {
        "guia_operadora": None,
        "beneficiario_codigo": None,
        "beneficiario_nome": None,
        "senha": None,
        "data_solicitacao": None,
    }

    with pdfplumber.open(pdf_path) as pdf:
        for pidx, page in enumerate(pdf.pages, start=1):
            buffer = ""
            text = page.extract_text() or ""
            
            for raw in text.splitlines():
                linha = normalize_line(raw)

                # Separadores
                m1 = RX_SEP1.search(linha)
                if m1:
                    # Flush buffer antes de mudar contexto
                    if buffer.strip():
                        parsed = parse_row_buffer(buffer)
                        if parsed:
                            registros.append({**contexto, **parsed, "_pagina": pidx, "_linha": buffer})
                        else:
                            falhas.append({"_pagina": pidx, "linha": buffer})
                        buffer = ""
                    
                    contexto["guia_operadora"] = m1.group(1)
                    contexto["beneficiario_codigo"] = m1.group(2)
                    contexto["beneficiario_nome"] = m1.group(3).strip()
                    continue

                m2 = RX_SEP2.search(linha)
                if m2:
                    # Flush buffer antes de mudar contexto
                    if buffer.strip():
                        parsed = parse_row_buffer(buffer)
                        if parsed:
                            registros.append({**contexto, **parsed, "_pagina": pidx, "_linha": buffer})
                        else:
                            falhas.append({"_pagina": pidx, "linha": buffer})
                        buffer = ""
                    
                    contexto["senha"] = m2.group(1)
                    contexto["data_solicitacao"] = m2.group(2)
                    continue

                # Linhas da tabela - ajustar para detectar "$" isolado também
                if RX_ROW_START.match(linha) or buffer:
                    buffer = (buffer + " " + linha).strip() if buffer else linha
                    
                    # Se linha terminou (tem "R$" ou "$"), processa
                    if RX_MONEY_END.search(buffer) or re.search(r'\$', buffer):
                        parsed = parse_row_buffer(buffer)
                        if parsed:
                            registros.append({**contexto, **parsed, "_pagina": pidx, "_linha": buffer})
                        else:
                            falhas.append({"_pagina": pidx, "linha": buffer})
                        buffer = ""

            # Flush final da página
            if buffer.strip():
                parsed = parse_row_buffer(buffer)
                if parsed:
                    registros.append({**contexto, **parsed, "_pagina": pidx, "_linha": buffer})
                else:
                    falhas.append({"_pagina": pidx, "linha": buffer})

    df = pd.DataFrame(registros)
    df_falhas = pd.DataFrame(falhas)
    return df, df_falhas

In [58]:
df, df_falhas = parse_pdf_text_as_table(pdf_path)
print("Linhas extraídas:", len(df))
print(f"Falhas restantes: {len(df_falhas)}")
if len(df_falhas) > 0:
    display(df_falhas.head(10))
else:
    print("✅ Nenhuma falha!")

display(df.head(20))

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

  [DEBUG] Cifrão isolado encontrado em: 1 00 40304361 - HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAÇÕES (ER1ITROGRAMA, L1EUCOGRAMA, PL1AQUETAS) - Exa0m1e/0s 7d/e2 0P2a5tologRia$ C9l,ín6i9ca
  [DEBUG] Parte após $: ' C9l,ín6i9ca'
  [DEBUG] Números antes da vírgula: ['9']
  [DEBUG] Números depois da vírgula: ['6', '9']
  [DEBUG] Valor reconstruído: 9,69
  [DEBUG] Cifrão isolado encontrado em: 9 00 40304361 - HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAÇÕES (ER1ITROGRAMA, L1EUCOGRAMA, PL1AQUETAS) - Exa0m1e/0s 7d/e2 0P2a5tologRia$ C9l,ín6i9ca
  [DEBUG] Parte após $: ' C9l,ín6i9ca'
  [DEBUG] Números antes da vírgula: ['9']
  [DEBUG] Números depois da vírgula: ['6', '9']
  [DEBUG] Valor reconstruído: 9,69
  [DEBUG] Cifrão isolado encontrado em: 7 00 40302750 - PERFIL LIPÍDICO / LIPIDOGRAMA (LIPÍDIOS TOTAIS, COLESTER1OL, TRIGLICERÍ1DIOS E ELETRO1FORESE LIPOPRO01T/E0Í7N/A2S0)2 5- ExaRm$e 3s 6d,e7 9Patologia Clínica
  [DEBUG] Parte após $: 'e 3s 6d,e7 9Patologia Clínica'
  [DEBUG] Números antes da

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Linhas extraídas: 1835
Falhas restantes: 0
✅ Nenhuma falha!


,guia_operadora,beneficiario_codigo,beneficiario_nome,senha,data_solicitacao,seq,tabela,procedimento_codigo,valor_exec,_pagina,_linha
0,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,20,00,40303136,10.52,1,"20 00 40303136 - SANGUE OCULTO, PESQUISA - Exa..."
1,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,19,00,40303110,10.52,1,19 00 40303110 - PARASITOLOGICO - Exames de Pa...
2,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,18,00,40302733,0.00,1,18 00 40302733 - HEMOGLOBINA GLICADA (FRAÇÃO A...
3,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,17,00,40302547,6.04,1,17 00 40302547 - TRIGLICERIDEOS - Exames de Pa...
4,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,16,00,40302695,8.03,1,16 00 40302695 - COLESTEROL (VLDL) - Exames de...
5,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,15,00,40301591,8.03,1,15 00 40301591 - COLESTEROL (LDL) - Exames de ...
6,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,14,00,40301583,6.04,1,14 00 40301583 - COLESTEROL (HDL) - Exames de ...
7,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,13,00,40301605,4.35,1,13 00 40301605 - COLESTEROL TOTAL - Exames de ...
8,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,12,00,40301150,4.35,1,12 00 40301150 - ACIDO URICO - Exames de Patol...
9,172381231,0001000190990004,IRACILDA CORREIA SANTOS,1723812312,01/07/2025,11,00,40311171,0.00,1,11 00 40311171 - MICROALBUMINURIA - Exames de ...


In [ ]:
df.groupby('beneficiario_nome')['valor_exec'].sum()

beneficiario_nome
ADRIANA ALEMAO SANTOS NOBRE              223.65
ALBANI DE MELO ROCHA SANTANA             149.86
ALBERTO BEDOIA DO NASCIMENTO             148.33
ALDANUZIA PEREIRA SILVA NASCIMENTO       114.40
ALEXANDRA PEREIRA NASCIMENTO              84.00
                                          ...  
VANUZIA MARIA DE JESUS DOS SANTOS         60.00
VERA LUCIA GONCALVES PEREIRA ASSUNCAO    171.40
VICENTE JOSE SOBRINHO                     89.63
VICTOR GABRIEL RIBEIRO NORONHA            60.00
ZILMA VIEIRA SANTANA SANTOS              157.38
Name: valor_exec, Length: 239, dtype: float64

: 

In [39]:
import re

def test_parse_line_simples(linha):
    """
    Parser simplificado: apenas Seq, Tabela, Procedimento (8 dígitos) e Valor Exec.
    Ignora todas as colunas intermediárias (quantidades, data exec)
    """
    
    # 1) Extrai seq, tabela, procedimento_codigo do início
    m_inicio = re.match(r'^\s*(\d+)\s+(\d{2})\s+(\d{8})', linha.strip())
    if not m_inicio:
        return None
    
    seq = int(m_inicio.group(1))
    tabela = m_inicio.group(2)
    proc_cod = m_inicio.group(3)
    
    # 2) Extrai valor (sempre presente, em qualquer lugar da linha)
    m_valor = re.search(r'R\$\s*([\d\.,]+)', linha)
    if not m_valor:
        return None
    
    valor = float(m_valor.group(1).replace(".", "").replace(",", "."))
    
    return {
        "seq": seq,
        "tabela": tabela,
        "procedimento_codigo": proc_cod,
        "valor_exec": valor
    }

# Teste com as linhas problemáticas
linhas_teste = [
    "18 00 40302733 - HEMOGLOBINA GLICADA (FRAÇÃO A1C) - Exames de Patologia C1línica 0 0 R$ 0,00",
    "17 00 40302547 - TRIGLICERIDEOS - Exames de Patologia Clínica 1 1 1 01/07/2025 R$ 6,04",
    "10 00 40311210 - ROTINA DE URINA (CARACTERES FISICOS, ELEMENTOS ANORM1AIS E SEDIMENT1OSCOPIA) - Exa1mes de Patologi0a1 C/0lín7i/c2a025 R$ 9,22"
]

for i, linha in enumerate(linhas_teste, 1):
    print(f"\n=== TESTE {i} ===")
    print(f"Linha: {linha}")
    resultado = test_parse_line_simples(linha)
    if resultado:
        print(f"Resultado: {resultado}")
    else:
        print("FALHA no parsing")


=== TESTE 1 ===
Linha: 18 00 40302733 - HEMOGLOBINA GLICADA (FRAÇÃO A1C) - Exames de Patologia C1línica 0 0 R$ 0,00
Resultado: {'seq': 18, 'tabela': '00', 'procedimento_codigo': '40302733', 'valor_exec': 0.0}

=== TESTE 2 ===
Linha: 17 00 40302547 - TRIGLICERIDEOS - Exames de Patologia Clínica 1 1 1 01/07/2025 R$ 6,04
Resultado: {'seq': 17, 'tabela': '00', 'procedimento_codigo': '40302547', 'valor_exec': 6.04}

=== TESTE 3 ===
Linha: 10 00 40311210 - ROTINA DE URINA (CARACTERES FISICOS, ELEMENTOS ANORM1AIS E SEDIMENT1OSCOPIA) - Exa1mes de Patologi0a1 C/0lín7i/c2a025 R$ 9,22
Resultado: {'seq': 10, 'tabela': '00', 'procedimento_codigo': '40311210', 'valor_exec': 9.22}


In [37]:
len('18 00 40302733 - HEMOGLOBINA GLICADA (FRAÇÃO A1C) - Exames de Patologia C1línica 0 0 R$ 0,00')

92